In [ ]:
import xml.etree.ElementTree as ET
import pyproj
from shapely.geometry import LineString, Polygon
from shapely.ops import transform
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def polygons(file_path):
    
    coordinates = []
    
    #PARSE THE KML FILE
    tree = ET.parse(file_path)
    root = tree.getroot()

    #FIND ALL POLYGON AND LINEARRING ELEMENTS
    for polygon in root.findall(".//{http://www.opengis.net/kml/2.2}Polygon"):
        for ring in polygon.findall("{http://www.opengis.net/kml/2.2}outerBoundaryIs/{http://www.opengis.net/kml/2.2}LinearRing"):
            #EXTRACT COORDINATES FROM THE COORDINATES ELEMENT
            coord_str = ring.find("{http://www.opengis.net/kml/2.2}coordinates").text
            coordinates.extend(coord_str.strip().split())

    coordinate_pairs = [tuple(map(float, coord.split(',')[:2])) for coord in coordinates]

    #CONVERT TO METERS COORDINATES:

    destine_proj = pyproj.Proj(init='epsg:31985')  #EPSG:31985 IS THE SIRGAS 2000 COORDINATES SYSTEM (USED FOR BRAZIL)
    polygon = Polygon(coordinate_pairs)
    projected_polygon = transform(destine_proj, polygon)

    #GET THE COORDINATES OF THE VERTICES
    perimeter = projected_polygon.exterior
    coordinates = list(perimeter.coords)

    #INSERT THE POLYGON
    polygon = coordinates

    effective_square_fetch = math.sqrt(projected_polygon.area)
    effective_circle_fetch = math.sqrt(4*abs(projected_polygon.area)/math.pi)
    print('Total area of polygon: %.2f m²' %projected_polygon.area)
    print('Effective square fetch: %.2f m' %effective_square_fetch)
    print('Effective circle fetch: %.2f m' %effective_circle_fetch)
    print('')
    
    return effective_square_fetch, effective_circle_fetch, polygon

In [ ]:
def fetch_calculations(wind_angle, qt_intervals, polygon):
    
    rot_polygon = []

    #SET WIND ANGLE
    angle = wind_angle

    #CHANGE ANGLE TO RADIANS
    rot_angle = (wind_angle) * math.pi / 180

    for i in range(0,len(polygon)):
        rot_polygon.append([polygon[i][0]*math.cos(rot_angle) + polygon[i][1]*math.sin(rot_angle),
                          -polygon[i][0]*math.sin(rot_angle) + polygon[i][1]*math.cos(rot_angle)])

    x_coords = [vertex[0] for vertex in rot_polygon]
    y_coords = [vertex[1] for vertex in rot_polygon]
    
    max_spacing = max(x_coords) - min(x_coords)

    #DEFINE NUMBER OF INTERVALS:
    intervals = qt_intervals
    spacing = max_spacing / intervals

    i = min(x_coords)
    intersections = []
    x_values = []
    poly = Polygon(rot_polygon)

    while(i <= max(x_coords)):
        #CREATING LINE AND POLYGON OBJECTS
        line_coords = [(i, min(y_coords)), (i, max(y_coords))]
        line = LineString(line_coords)

        #VERIFY AND STORE INTERSECTIONS BETWEEN LINE AND POLYGON
        full_intersections = []
        for k in range(len(rot_polygon)):
            side_coords = [rot_polygon[k], rot_polygon[(k+1)%len(rot_polygon)]]
            side = LineString(side_coords)
            if line.intersects(side):
                intersection = line.intersection(side)
                if intersection.geom_type == 'Point':
                    intersections.append((i,intersection.x, intersection.y))

        number = i
        bigger_numbers = [x for x in x_coords if x > number]
        if(i == max(x_coords)):
            i = max(x_coords) + 1
        else:
            if bigger_numbers:
                next_vertex = min(bigger_numbers, key=lambda x:abs(x - number))

            if(next_vertex < i + spacing):
                i = next_vertex
                x_values.append(i)

            else:
                i += spacing
                x_values.append(i)

    df = pd.DataFrame(data=intersections)
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    df.columns = ['x_value','intersec_x','intersec_y']

    #FIND UNIQUE VALUES IN THE X_VALUE COLUMN
    total_area = 0
    unique_values = df['x_value'].unique()
    areas_list = []

    #ITERATE THROUGH THE UNIQUE VALUES TO CREATE PAIRS
    for i in range(len(unique_values) - 1):
        pair = [unique_values[i], unique_values[i + 1]]
        indices = df[df['x_value'].isin(pair)].index
        temp_df = df.loc[indices]
        temp_df = temp_df.drop(columns=['x_value'])

        combined_lists = temp_df.apply(lambda row: row.tolist(), axis=1)
        temp_list = combined_lists.tolist()

        area = 0

        if(len(temp_list) == 3):
            for j in range(len(temp_list)):
                x1, y1 = temp_list[j]
                x2, y2 = temp_list[(j + 1) % len(temp_list)]
                formula_value = (x1 * y2 - x2 * y1)
                area += formula_value/2

        elif(len(temp_list) == 4):
            x1, y1 = temp_list[0]
            x2, y2 = temp_list[1]
            x3, y3 = temp_list[2]
            x4, y4 = temp_list[3]

            h = x3-x1
            b = (abs(y1-y2)+abs(y3-y4))/2
            area = b*h

        areas_list.append(abs(area))
        Polygon(temp_list)
        total_area += abs(area)

    #CALCULATE THE INCRIMENT GIVEN AT EVERY POLYGON (WIDTH OF POLYGON)
    increments = []
    for i in range(len(x_values)):
        if i == 0:
            increments.append(x_values[i] - min(x_coords))
        else:
            increments.append(x_values[i] - x_values[i-1])

    #STORE DATA INTO DATAFRAME

    polygons_df = pd.DataFrame(areas_list, columns=['Area (m²)'])
    new_index_labels = ['Polygon {}'.format(i + 1) for i in range(len(polygons_df))]
    polygons_df.index = new_index_labels

    polygons_df['Width (m)'] = increments

    #CALCULATING THE FETCH OF EACH POLYGON
    polygons_df['Fetch (m)'] = polygons_df['Area (m²)'] / polygons_df['Width (m)']
    polygons_df
    
    fetch_values = polygons_df['Fetch (m)'].tolist()
    area_values = polygons_df['Area (m²)'].tolist()
    
    return fetch_values, area_values

In [ ]:
def calculations(data_qty, z0, ScL, dataset, polygon_data2):
    A = -0.052 * z0**0.066 + 0.051
    b = 1.24 + 4.1*math.log(z0)*10**(-3)
    c = 0.5*z0**0.38
    columns = ['Intervals', 'Wind Direction', 'Wind Velocity', 'Friction Velocity (u*)','Effective Square u*', 'Effective Circle u*', 'Square u* Variation', 'Circle u* Variation', 'kL', 'Effective Square kL', 'Effective Circle kL', 'Square kL Variation', 'Circle kL Variation']
    
    calculations_df = pd.DataFrame(columns=columns)
    
    for i in range(0, data_qty):
        friction_values = []
        kl_values =[]
        u_values = []
        fetch_list, area_list = fetch_calculations(dataset['Wind direction'][i], inters, polygon_data2)
        
        for j in range(len(fetch_list)):
            u = A * (dataset['Wind velocity'][i]**b) * (fetch_list[j]**c)
            
            if (fetch_list[j]>=16):
                kL = 4.31 * 10**(-3) * u * ScL**(-0.5)
            elif (fetch_list[j]!=0):
                kL = (1.191 + 2.551 * math.log(fetch_list[j])) * 10**(-3) * u * ScL**(-0.5)
            else:
                pass
                
            friction_values.append(u*area_list[j])
            kl_values.append(kL*area_list[j])
            u_values.append(u**2*area_list[j])
            
        hour_friction_mean = sum(friction_values) / sum(area_list)
        hour_kl_mean = sum(kl_values) / sum(area_list)
        u_mean = np.sqrt(sum(u_values) / sum(area_list))
        
        print("---------------------------------------------------------------")
        print('kL value for', inters,'intervals, at wind direction',dataset['Wind direction'][i],'degrees and wind velocity',dataset['Wind velocity'][i],'m/s:', hour_kl_mean)
        print('')
        
        u2 = A * (dataset['Wind velocity'][i]**b) * (effective_square_fetch**c)
       
        kL2 = 4.31 * 10**(-3) * u2 * ScL**(-0.5)
        print('Effective square calculated kL:', kL2)
        print('Variation:', (kL2-hour_kl_mean)/abs(hour_kl_mean)*100,'%')
        print('')

        u3 = A * (dataset['Wind velocity'][i]**b) * (effective_circle_fetch**c)
        
        kL3 = 4.31 * 10**(-3) * u3 * ScL**(-0.5)
        print('Effective  calculated kL:', kL3)
        print('Variation:', (kL3-hour_kl_mean)/abs(hour_kl_mean)*100,'%')
        print('')
        print("---------------------------------------------------------------")
        
        all_data = {'Intervals': [inters], 'Wind Direction': [dataset['Wind direction'][i]], 'Wind Velocity': [dataset['Wind velocity'][i]], 'Friction Velocity (u*)':[u_mean],'Effective Square u*': [u2], 'Effective Circle u*': [u3], 'Square u* Variation': [(u2-u_mean)/abs(u_mean)], 'Circle u* Variation': [(u3-u_mean)/abs(u_mean)], 'kL': [hour_kl_mean], 'Effective Square kL': [kL2], 'Effective Circle kL': [kL3], 'Square kL Variation': [(kL2-hour_kl_mean)/abs(hour_kl_mean)], 'Circle kL Variation': [(kL3-hour_kl_mean)/abs(hour_kl_mean)]}
        calculations_df = calculations_df.append(pd.DataFrame(all_data), ignore_index=True)
        
    return calculations_df

In [ ]:
def generic_polygon(polygon_coordinates):
    coordinates = []
    
    polygon3 = Polygon(polygon_coordinates)

    effective_square_fetch = math.sqrt(polygon3.area)
    effective_circle_fetch = math.sqrt(4*abs(polygon3.area)/math.pi)
    print('Total area of polygon: %.2f m²' %polygon3.area)
    print('Effective square fetch: %.2f m' %effective_square_fetch)
    print('Effective circle fetch: %.2f m' %effective_circle_fetch)
    print('')
    
    return effective_square_fetch, effective_circle_fetch, polygon_coordinates

In [ ]:
#SET THE PATH TO THE WEATHER DATA (MUST CONTAIN WIND DIRECTION AND WIND VELOCITY COLUMNS)

weather_data = 

In [ ]:
####GOOGLE EARTH DEFINED SURFACE####

#SET THE KML PATH OF THE SURFACE:
file_path = 

#DEFINE THE NUMBER OF INTERVALS:
inters = 100

#DATA QUANTITY TO ANALISE:
data_qty = len(weather_data)

#CALCULATIONS PARAMETERS:
z0 = 0.1
ScL = 594

#FUNCTIONS CALLS:
effective_square_fetch, effective_circle_fetch, polygon2 = polygons(file_path)
final_df = calculations(data_qty, z0, ScL, year_data, polygon2)

######EXTRACT DATA TO EXCEL######
#DEFINE PATH TO THE SAVE
excel_path = 
final_df.to_excel(excel_path, index=False)

In [ ]:
#####MATHEMATICAL SURFACE#####

#DEFINE THE POLYGON VERTICES
polygon_data = [(0,0),(136.51,0),(136.51,136.51),(0,136.51)]

#NUMBER OF INTERVALS:
inters = 100

#DATA QUANTITY TO ANALISE:
data_qty = len(weather_data)

#CALCULATIONS PARAMETERS:
z0 = 0.1
ScL = 594

#FUNCTIONS CALLS:
effective_square_fetch, effective_circle_fetch, polygon2 = generic_polygon(polygon_data)
final_df = calculations(data_qty, z0, ScL, year_data, polygon2)

######EXTRACT DATA TO EXCEL######
#DEFINE PATH TO THE SAVE
excel_path = 
final_df.to_excel(excel_path, index=False)